## 분류
### MNIST 데이터셋,  머신러닝 분야의 'Hello world'

In [ ]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "classification"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

### 데이터셋 불러오기

openml.org에 접속해서 데이터셋을 불러오기 때문에 시간이 걸립니다.

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame = False)
print(type(mnist))

In [ ]:
X, y = mnist['data'], mnist['target']
X.shape

In [ ]:
y.shape

In [ ]:
28*28

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

some_digit = X[0]
some_digit_image = some_digit.reshape(28,28)
plt.imshow(some_digit_image, cmap = matplotlib.cm.binary,
          interpolation='nearest')
#save_fig("some_digit_plot")
plt.axis('off')
plt.show()

In [ ]:
y[0]

타켓(클래스)의 데이터 타입이 문자열이므로 숫자형태로 변환해 줌

In [ ]:
y = y.astype(np.int64)

In [ ]:
y[0]

데이터셋의 X 데이터들이 어떤 이미지인지 확인하기 위해 아래 코드를 실행

In [ ]:
def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap = mpl.cm.binary,
               interpolation="nearest")
    plt.axis("off")
    
  # 숫자 그림을 위한 추가 함수
def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = mpl.cm.binary, **options)
    plt.axis("off")  
    
plt.figure(figsize=(9,9))
example_images = X[:100]
plot_digits(example_images, images_per_row=10)
save_fig("more_digits_plot")
plt.show()

### 훈련 데이터 60,000개,  테스트데이터 10,000개  분리

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

### 5인지 아닌지를 분류
- target 값을 True(5인 경우)와 False(5가 아닌 경우)로 변경

In [ ]:
y_train_5  = (y_train == 5)
y_test_5 = (y_test == 5)

In [ ]:
y_train_5[10:50]

### SGD Classfier(분류기)에 의해 학습

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42)
sgd_clf.fit(X_train, y_train_5)

- some_digit (X[0])를 예측

In [ ]:
sgd_clf.predict([some_digit])

### 교차검증을 이용한 정확도 측정
- 정확도 : 전체 데이터 중 제대로 예측한 건수

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring='accuracy')

### 평균 정확도가 95% 이상이므로 모델의 성능이 매우 좋다??

- 비교를 위해 무조건 5가 아니라고 분류하는 분류기(더미 분류기)를 만들어 적용해 보기

In [ ]:
from sklearn.base import BaseEstimator
class Never5Classifier(BaseEstimator):
    def fit(self, X, y = None):
        pass
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

In [ ]:
never_5_clf = Never5Classifier()
cross_val_score(never_5_clf, X_train,y_train_5, cv=3, scoring='accuracy')

- 이미지의 10% 정도만 숫자 5이기 때문에, 무조건 '5아님'으로 예측해도 정확도가 90%
- 불균형 데이터셋에서는 정확도가 성능 측정 지표로 부적합

### 분류의 성능 평가 - PPT 124 참조

In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)  # 교차검증에 의한 예측값 반환
y_train_pred[:10]

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train_5, y_train_pred)  # 오차행렬, (이진)분류의 예측이 얼마나 잘 수행되고 있는지를 보여줌

In [ ]:
y_train_perfect_predictions = y_train_5  # 예측 결과가 학습데이터의 타겟값과 같다고 가정하면,
confusion_matrix(y_train_5, y_train_perfect_predictions)

- 정밀도 : positive로 예측한 대상 중에 TP(True Positive)의 비율
  * TP(진짜 양성)를 높이는 것이 목표
  * NP(가짜 양성)를 낮추는 것이 목표
  * 스팸 메일(스팸이면 양성)을 분류할 때, 스팸이 아닌 메일을 스팸으로 분류하게 되면 메일을 받을 수 없음, 이런 경우에는 정밀도가 중요

In [ ]:
from sklearn.metrics import precision_score, recall_score

precision_score(y_train_5, y_train_pred)

In [ ]:
cm = confusion_matrix(y_train_5, y_train_pred)   # 실제 계산에 의해 산출
cm[1, 1] / (cm[0, 1] + cm[1, 1])

- 재현율 : 실제 positive 중에 TP(True Positive)의 비율
  * TP(진짜 양성)를 높이는 것이 목표
  * FN(가짜 음성)를 낮추는 것이 목표
  * 암 판단 모델, 금융사기적발 모델과 같이 실제 양성 데이터를 음성으로 잘못 판단하면 업무상 영향이 큰 경우에 성능기준으로 활용

In [ ]:
recall_score(y_train_5, y_train_pred)

In [ ]:
cm[1, 1] / (cm[1, 0] + cm[1, 1]) # 실제 계산에 의해 산출

- F1 점수
  * 정밀도와 재현율은 보완적인 지표이므로, 둘 다 높은 수치인 경우가 가장 좋은 성능을 의미
  * 정밀도와 재현율의 조화 평균  


In [ ]:
from sklearn.metrics import f1_score

f1_score(y_train_5, y_train_pred)

In [ ]:
cm[1, 1] / (cm[1, 1] + (cm[1, 0] + cm[0, 1]) / 2)

In [ ]:
#y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3, method='decision_function')
y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)
y_scores[:10]

**classification_report()**<br>
 분류 모델의 예측 결과와 실제 클래스 레이블을 기반으로 다양한 분류 지표들을 계산하여 보고서 형태로 제공

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train_5, y_scores))

- ROC 곡선
  * FPR(거짓양성비율)에 대한 TPR(진짜양성비율, 재현율)의 곡선
  * roc_curve() 함수로 곡선 계산하고, roc_auc_score() 함수로 면적 계산

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--') # 대각 점선
    plt.xlabel('False Positive Rate (Fall-Out)', fontsize=16) # Not shown
    plt.ylabel('True Positive Rate (Recall)', fontsize=16)    # Not shown
    plt.grid(True)                                            # Not shown

plt.figure(figsize=(8, 6))                                    # Not shown
plot_roc_curve(fpr, tpr)
save_fig("roc_curve_plot")                                    # Not shown
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_train_5, y_scores)

### RandomForestClassifier에 의한 학습

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
y_train_pred_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3)
y_train_pred_forest[:10]

In [ ]:
y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3,
                                    method="predict_proba")

In [ ]:
y_scores_forest = y_probas_forest[:, 1] # 점수 = 양성 클래스의 확률
y_scores_forest[:10]

In [ ]:
precision_score(y_train_5, y_train_pred_forest)

In [ ]:
recall_score(y_train_5, y_train_pred_forest)

In [ ]:
f1_score(y_train_5, y_train_pred_forest)

In [ ]:
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_5,y_scores_forest)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, "b:", linewidth=2, label="SGD")
plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")

plt.grid(True)
plt.legend(loc="lower right", fontsize=16)
save_fig("roc_curve_comparison_plot")
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train_5, y_scores_forest)

### SGD Classifier, RandomForest Classifier 성능 비교

           SGD   RandomForest 
                     
  정밀도(precision)      0.84       0.99
  
  재현율(recall)         0.65       0.86    
  
  F1                      0.73     0.92
  
  ROC Auc                 0.96     0.99
